# Vytvoření regresního modelu: příprava a vizualizace dat

## **Lineární regrese pro dýně - Lekce 2**
#### Úvod

Nyní, když máte k dispozici nástroje potřebné k zahájení budování modelů strojového učení pomocí Tidymodels a Tidyverse, jste připraveni začít klást otázky svým datům. Při práci s daty a aplikaci řešení strojového učení je velmi důležité vědět, jak položit správnou otázku, abyste mohli plně využít potenciál svého datasetu.

V této lekci se naučíte:

-   Jak připravit data pro budování modelu.

-   Jak používat `ggplot2` pro vizualizaci dat.

Otázka, na kterou potřebujete odpověď, určí, jaký typ algoritmů strojového učení budete používat. Kvalita odpovědi, kterou získáte, bude silně záviset na povaze vašich dat.

Podívejme se na to prostřednictvím praktického cvičení.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Ilustrace od @allison_horst</figcaption>


<!--![Ilustrace od \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Ilustrace od \@allison_horst-->


## 1. Importování dat o dýních a přivolání Tidyverse

Budeme potřebovat následující balíčky, abychom mohli rozdělit a zpracovat tuto lekci:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [sbírka balíčků pro R](https://www.tidyverse.org/packages), která je navržena tak, aby byla datová věda rychlejší, jednodušší a zábavnější!

Můžete je nainstalovat pomocí:

`install.packages(c("tidyverse"))`

Níže uvedený skript zkontroluje, zda máte balíčky potřebné k dokončení tohoto modulu, a v případě, že některé chybí, je pro vás nainstaluje.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Nyní spusťme některé balíčky a načtěme [data](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) poskytnutá pro tuto lekci!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Rychlý `glimpse()` okamžitě ukazuje, že jsou zde prázdné hodnoty a směs řetězců (`chr`) a číselných dat (`dbl`). `Date` je typu znakového řetězce a je tu také zvláštní sloupec nazvaný `Package`, kde jsou data směsí mezi `sacks`, `bins` a dalšími hodnotami. Data jsou vlastně trochu chaotická 😤.

Ve skutečnosti není příliš běžné dostat dataset, který je zcela připravený k použití pro vytvoření ML modelu hned na první pokus. Ale nezoufejte, v této lekci se naučíte, jak připravit surový dataset pomocí standardních knihoven v R 🧑‍🔧. Také se naučíte různé techniky vizualizace dat.📈📊
<br>

> Připomenutí: Operátor pipe (`%>%`) provádí operace v logickém sledu tím, že předává objekt dál do funkce nebo výrazu. Můžete si operátor pipe představit jako "a potom" ve vašem kódu.


## 2. Kontrola chybějících dat

Jedním z nejčastějších problémů, se kterými se datoví vědci musí vypořádat, jsou neúplná nebo chybějící data. R reprezentuje chybějící nebo neznámé hodnoty pomocí speciálního sentinelového hodnoty: `NA` (Not Available).

Jak tedy zjistíme, že datový rámec obsahuje chybějící hodnoty?
<br>
-   Jedním z nejjednodušších způsobů je použití základní funkce R `anyNA`, která vrací logické objekty `TRUE` nebo `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Skvělé, zdá se, že některá data chybí! To je dobré místo, kde začít.

-   Dalším způsobem by bylo použít funkci `is.na()`, která označuje, které jednotlivé prvky sloupce chybí pomocí logické hodnoty `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Dobře, hotovo, ale s tak velkým datovým rámcem, jako je tento, by bylo neefektivní a prakticky nemožné zkontrolovat všechny řádky a sloupce jednotlivě😴.

-   Intuitivnější způsob by byl spočítat součet chybějících hodnot pro každý sloupec:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Mnohem lepší! Chybí některá data, ale možná to nebude mít vliv na daný úkol. Uvidíme, jaký další rozbor přinese.

> Spolu s úžasnými balíčky a funkcemi má R velmi dobrou dokumentaci. Například použijte `help(colSums)` nebo `?colSums`, abyste se dozvěděli více o této funkci.


## 3. Dplyr: Gramatika manipulace s daty

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Ilustrace od @allison_horst</figcaption>


<!--![Ilustrace od \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Ilustrace od \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), balíček v Tidyverse, je gramatikou manipulace s daty, která poskytuje konzistentní sadu sloves, jež vám pomohou vyřešit nejběžnější výzvy při manipulaci s daty. V této části se podíváme na některá slovesa z dplyr!  
<br>


#### dplyr::select()

`select()` je funkce z balíčku `dplyr`, která vám pomáhá vybrat sloupce, které chcete zachovat nebo vyloučit.

Abyste si usnadnili práci s datovým rámcem, odstraňte několik jeho sloupců pomocí `select()` a ponechte pouze ty, které potřebujete.

Například v tomto cvičení bude naše analýza zahrnovat sloupce `Package`, `Low Price`, `High Price` a `Date`. Pojďme tyto sloupce vybrat.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` je funkce z balíčku `dplyr`, která vám umožňuje vytvářet nebo upravovat sloupce, přičemž zachovává stávající sloupce.

Obecná struktura funkce mutate je:

`data %>%   mutate(new_column_name = what_it_contains)`

Pojďme si vyzkoušet `mutate` na sloupci `Date` provedením následujících operací:

1.  Převést data (aktuálně typu znak) na formát měsíce (jedná se o americká data, takže formát je `MM/DD/YYYY`).

2.  Extrahovat měsíc z dat do nového sloupce.

V R balíček [lubridate](https://lubridate.tidyverse.org/) usnadňuje práci s daty a časy. Takže použijeme `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` a podíváme se, jak dosáhnout výše uvedených cílů. Sloupec Date můžeme vynechat, protože ho v dalších operacích nebudeme potřebovat.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Hurá! 🤩

Nyní vytvořme nový sloupec `Price`, který bude představovat průměrnou cenu dýně. Vezmeme průměr hodnot ze sloupců `Low Price` a `High Price` a použijeme jej k naplnění nového sloupce Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

„Ale počkejte!“, řeknete po rychlém prohlédnutí celého datového souboru pomocí `View(pumpkins)`, „Tady je něco zvláštního!“🤔

Pokud se podíváte na sloupec `Package`, dýně se prodávají v mnoha různých konfiguracích. Některé se prodávají v mírách `1 1/9 bushel`, některé v mírách `1/2 bushel`, některé na kus, některé na váhu, a některé ve velkých krabicích s různými šířkami.

Pojďme si to ověřit:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Úžasné!👏

Zdá se, že dýně je velmi těžké vážit konzistentně, takže je vyfiltrujeme tím, že vybereme pouze dýně, které obsahují řetězec *bushel* ve sloupci `Package`, a uložíme je do nového datového rámce `new_pumpkins`.


#### dplyr::filter() a stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): vytváří podmnožinu dat obsahující pouze **řádky**, které splňují vaše podmínky, v tomto případě dýně s řetězcem *bushel* ve sloupci `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): zjišťuje přítomnost nebo nepřítomnost vzoru v řetězci.

Balíček [`stringr`](https://github.com/tidyverse/stringr) poskytuje jednoduché funkce pro běžné operace s řetězci.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Můžete vidět, že jsme zúžili výběr na přibližně 415 řádků dat obsahujících dýně po bušlech.🤩  
<br>


#### dplyr::case_when()

**Ale počkejte! Ještě je třeba něco udělat**

Všimli jste si, že množství v bušlech se liší podle řádku? Je potřeba normalizovat ceny tak, aby ukazovaly cenu za bušel, a ne za 1 1/9 nebo 1/2 bušle. Je čas provést nějaké výpočty, abychom to standardizovali.

Použijeme funkci [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) k *úpravě* sloupce Price na základě určitých podmínek. `case_when` umožňuje vektorizovat více `if_else()` výroků.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Nyní můžeme analyzovat cenu za jednotku na základě jejich měření v bušlech. Celá tato studie bušlů dýní však ukazuje, jak velmi `důležité` je `pochopit povahu vašich dat`!

> ✅ Podle [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) závisí váha bušle na typu produktu, protože jde o objemové měření. "Bušl rajčat by například měl vážit 56 liber... Listy a zelené rostliny zabírají více prostoru s menší váhou, takže bušl špenátu váží pouze 20 liber." Je to všechno docela složité! Nezabývejme se převodem bušlů na libry a místo toho stanovme cenu podle bušle. Celá tato studie bušlů dýní však ukazuje, jak velmi důležité je pochopit povahu vašich dat!
>
> ✅ Všimli jste si, že dýně prodávané na půl bušle jsou velmi drahé? Dokážete zjistit proč? Nápověda: malé dýně jsou mnohem dražší než velké, pravděpodobně proto, že jich je mnohem více na bušl, vzhledem k nevyužitému prostoru, který zabírá jedna velká dutá dýně na koláč.


Nyní nakonec, čistě pro dobrodružství 💁‍♀️, pojďme také přesunout sloupec Month na první pozici, tj. `před` sloupec `Package`.

Funkce `dplyr::relocate()` se používá ke změně pozic sloupců.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Dobrá práce!👌 Nyní máte čistý a uspořádaný dataset, na kterém můžete postavit svůj nový regresní model!  
<br>


## 4. Vizualizace dat s ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


<!--![Infografika od Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Existuje *moudré* rčení, které říká:

> "Jednoduchý graf přinesl analytikovi více informací než jakékoli jiné zařízení." --- John Tukey

Součástí role datového vědce je ukázat kvalitu a povahu dat, se kterými pracuje. K tomu často vytváří zajímavé vizualizace, nebo grafy, diagramy a tabulky, které zobrazují různé aspekty dat. Tímto způsobem mohou vizuálně ukázat vztahy a mezery, které by jinak bylo obtížné odhalit.

Vizualizace také mohou pomoci určit, která technika strojového učení je pro daná data nejvhodnější. Například bodový graf, který se zdá sledovat přímku, naznačuje, že data jsou vhodným kandidátem pro úlohu lineární regrese.

R nabízí několik systémů pro tvorbu grafů, ale [`ggplot2`](https://ggplot2.tidyverse.org/index.html) je jedním z nejelegantnějších a nejvšestrannějších. `ggplot2` vám umožňuje skládat grafy **kombinováním nezávislých komponent**.

Začněme jednoduchým bodovým grafem pro sloupce Price a Month.

V tomto případě začneme s [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), dodáme dataset a estetické mapování (pomocí [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) a poté přidáme vrstvy (jako [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) pro bodové grafy.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Je tento graf užitečný 🤷? Překvapilo vás na něm něco?

Není příliš užitečný, protože pouze zobrazuje vaše data jako rozptyl bodů v daném měsíci.  
<br>


### **Jak to udělat užitečné?**

Aby grafy zobrazovaly užitečná data, obvykle je potřeba data nějakým způsobem seskupit. Například v našem případě by nalezení průměrné ceny dýní za každý měsíc poskytlo více informací o základních vzorcích v našich datech. To nás přivádí k dalšímu rychlému přehledu **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Seskupenou agregaci v R lze snadno vypočítat pomocí

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` mění jednotku analýzy z celého datasetu na jednotlivé skupiny, například podle měsíců.

-   `dplyr::summarize()` vytváří nový datový rámec s jedním sloupcem pro každou seskupovací proměnnou a jedním sloupcem pro každou zadanou statistiku shrnutí.

Například můžeme použít `dplyr::group_by() %>% summarize()` k seskupení dýní do skupin na základě sloupce **Month** a následně najít **průměrnou cenu** za každý měsíc.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Stručně!✨

Kategorie, jako jsou měsíce, se lépe zobrazují pomocí sloupcového grafu 📊. Vrstvy, které se používají pro sloupcové grafy, jsou `geom_bar()` a `geom_col()`. Podívejte se na `?geom_bar`, abyste zjistili více.

Pojďme si jeden vytvořit!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Toto je užitečnější vizualizace dat! Zdá se, že naznačuje, že nejvyšší cena za dýně se vyskytuje v září a říjnu. Odpovídá to vašemu očekávání? Proč ano nebo proč ne?

Gratulujeme k dokončení druhé lekce 👏! Připravili jste svá data pro tvorbu modelu a poté odhalili další poznatky pomocí vizualizací!



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
